# **Students Names: Mariem Aalabou & Emmarius Delar**

# MCS Algorithms for Order and Chaos Game

### Useful Links
- [Order and Chaos - Wikipedia](https://en.wikipedia.org/wiki/Order_and_Chaos)
- [Order and Chaos - Game simulator](https://ludii.games/details.php?keyword=Order%20and%20Chaos)

**Abstract :** For this project we interested about Order and Chaos game who is very challenging because of is rules. Like in tic-tac-toe players allowed between to state O or X. His particularity come from the fact that both players allowed to play O and X, so the main objective for player 1 and player 2 are different. The first want to make a winning line of 5 successivly same symbole and the other want to make a draw.

## Import

In [ ]:
import numpy as np
import numpy.random as random
from copy import deepcopy
import math
from collections import defaultdict
import matplotlib.pyplot as plt
import time

In [ ]:
from IPython.display import display, clear_output, HTML
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import random
from copy import deepcopy
import seaborn as sns
import base64

## Constant inputs

In [ ]:
Dx = 6
Dy = 6
Empty = '-'
# Create the Zobrist hashing table
HashTable = [[ np.random.randint(0, 2**15, Dy).tolist() for _ in range(Dx)] for _ in range(2)]
HashTurn = np.random.randint(0, 2**15)

## Define the Game

### Move class

In [ ]:
class Move(object):
    def __init__(self, x, y, symbol):
        self.x = x
        self.y = y
        self.symbol = symbol

    def valid (self, board):
        # Move in the board
        if self.x >= Dx or self.y >= Dy or self.x < 0 or self.y <0:
            return False
        # Move in a free space
        if board.board[self.x][self.y] != Empty:
            return False

        return True

### Class Board

In [ ]:
class Board(object):
    def __init__(self):
        #self.history = {'Order' : [], 'Chaos' : []}
        self.turn = 'Order' #Order always start the game
        self.board = [[Empty for _ in range(Dx)] for _ in range(Dy)]
        self.board = np.array(self.board)
        self.nb_moves = 0
        self.h = 0

    def legalMoves(self):
        moves = []
        for i in range (0, Dx):
            for j in range (0, Dy):
                for symb in ['X', 'O']:
                    m = Move (i, j, symb)
                    if m.valid (self):
                        moves.append (m)

        return moves

    def NearLegalMoves(self, deg=1):
        moves = []
        for played_m in self.move_history:
            for i in range (-deg, deg+1):
                for j in range (-deg, deg+1):
                    if i == 0 and j == 0:
                        pass
                    else:
                        for symb in ['X', 'O']:
                            coord_x = played_m.x
                            coord_y = played_m.y
                            m = Move (coord_x+i, coord_y+j, symb)
                            if m.valid (self):
                                moves.append (m)

        return moves

    def verif_array(self, array):
        if (np.char.count(array, '-').sum() <= 1) and (len(np.unique(array[:5]))==1 or len(np.unique(array[1:]))==1):
            return True
        else:
            return False

    def win(self):
        # Horizontal win
        for row in self.board:
            if self.verif_array(row) == True:
                return True

        # Vertical win
        for col in self.board.T:
            if self.verif_array(col) == True:
                return True

        # Diagonal win
        for i in [-1, 0, 1]:
            diag = np.diagonal(self.board, offset=i)
            if (len(diag)==6) and (self.verif_array(diag) == True) or (np.char.count(diag, '-').sum() < 1) and (len(np.unique(diag))==1):
                return True

            diag_opposite = np.flipud(self.board).diagonal(offset=i)
            if (len(diag_opposite)==6) and (self.verif_array(diag_opposite) == True) or (np.char.count(diag_opposite, '-').sum() < 1) and (len(np.unique(diag_opposite))==1):
                return True

        return False


    def draw(self):
        if self.nb_moves < Dx*Dy:
            return False
        if self.nb_moves >= Dx*Dy and not self.win():
            return True
        return False

    def score(self):
        if self.draw():
            return 0
        if self.win():
            return 1

    def terminal(self):
        if not self.win() and not self.draw():
            return False
        return True

    def __repr__(self):
        return "\n".join(" ".join(row) for row in self.board)

    def play (self, move:Move):
        change_player = {'Order' : 'Chaos', 'Chaos' : 'Order'}
        encode_symbol = {'X' : 0, 'O' : 1}
        if move.valid(self):
            self.board[move.x][move.y] = move.symbol
            # Compute hash
            self.h = self.h ^ HashTable[ encode_symbol[move.symbol] ][move.x][move.y]
            self.h = self.h ^ HashTurn
            # Actualise
            self.turn = change_player[self.turn]
            self.nb_moves += 1

    def playout (self):
        while (True):
            moves = self.legalMoves()
            if self.terminal():
                return self.score()
            n = random.randint(0, len (moves) - 1)
            self.play(moves [n])

## Test the game

### Manually

In [ ]:
game = Board()
game.play(Move(0, 0, 'X'))
game.play(Move(1, 1, 'X'))
game.play(Move(2, 2, 'O'))
game.play(Move(3, 3, 'X'))
game.play(Move(4, 4, 'X'))
print(game)
print("Win:", game.win())
print("Draw:", game.draw())
print("End:", game.terminal())
game.play(Move(2, 0, 'O'))
game.play(Move(2, 1, 'O'))
game.play(Move(2, 3, 'O'))
game.play(Move(2, 4, 'O'))
print(game)
print("Win:", game.win())
print("Draw:", game.draw())
print("End:", game.terminal())

### Playout

In [ ]:
game = Board()
game.playout()
print(game)
print("Win:", game.win())
print("Draw:", game.draw())
print("End:", game.terminal())

### Win rate after 1000 playout

In [ ]:
wins = []
for _ in range(1000):
    game = Board()
    score = game.playout()
    wins.append(score)

print(f"Win rate after 1000 playout: {sum(wins)/1000:.4f}")

# Define algorithms

## UCT

In [ ]:
Table = {}
MaxLegalMoves = 2*Dx*Dy

def add(board):
    nplayouts = [0.0 for x in range(MaxLegalMoves)]
    nwins = [0.0 for x in range(MaxLegalMoves)]
    Table[board.h] = [0, nplayouts, nwins]

def look(board):
    return Table.get(board.h, None)

def UCT(board, c=1.41):
    moves = board.legalMoves()
    if board.terminal():
        return board.score()

    t = look(board)
    if t != None:
        bestValue = -1
        bestMove = moves[0]
        i_best = 0

        n, p, w = t
        for i, m in enumerate(moves):
            p_m = p[i]
            w_m = w[i]
            if p_m > 0:
                value = np.abs((board.turn == 'Chaos') - (w_m/p_m)) + c * np.sqrt(np.log(n)/p_m)
                if value > bestValue:
                    i_best = i
                    bestValue = value
                    bestMove = m

        b = deepcopy(board)
        b.play(bestMove)
        res = b.playout()
        res = np.abs((board.turn == 'Chaos') - res)
        t[0] += 1
        t[1][i_best] += 1
        t[2][i_best] += res
        return res
    else:
        add(board)
        b = deepcopy(board)
        rdm_move = random.randint(0, len(moves) - 1)
        b.play(moves[rdm_move])
        res = b.playout()
        res = np.abs((b.turn == 'Chaos') - res)
        t = look(board)
        t[0] += 1
        t[1][rdm_move] += 1
        t[2][rdm_move] += res
        return res

def BestMoveUCT(board, iterations=100):
    global Table
    Table.clear()
    for i in range(iterations):
        b = deepcopy(board)
        UCT(b)

    t = look(board)
    if t is None:
        moves = board.legalMoves()
        return moves[random.randint(0, len(moves) - 1)]

    moves = board.legalMoves()
    if not moves:
        return None

    bestMove = moves[0]
    bestValue = t[1][0]

    for i in range(1, len(moves)):
        if i < len(t[1]) and t[1][i] > bestValue:
            bestValue = t[1][i]
            bestMove = moves[i]

    return bestMove

## Enhanced Playout Methods

In [ ]:
def heuristic_playout(board):
    """Heuristic playout that prioritizes winning moves for Order and blocking moves for Chaos"""
    board_copy = deepcopy(board)
    while not board_copy.terminal():
        winning_move = find_tactical_move(board_copy)
        if winning_move:
            board_copy.play(winning_move)
            continue

        moves = board_copy.legalMoves()
        if not moves:
            break
        n = random.randint(0, len(moves) - 1)
        board_copy.play(moves[n])

    if board_copy.terminal():
        return board_copy.score()
    else:
        return 0

In [ ]:
def find_tactical_move(board):
    """Trying to find winning move for Order or blocking move for Chaos"""
    moves = board.legalMoves()

    if board.turn == 'Order':
        for move in moves:
            temp_board = deepcopy(board)
            temp_board.play(move)
            if temp_board.win():
                return move

    else:
        next_board = deepcopy(board)
        next_board.turn = 'Order'
        order_moves = []
        for i in range(Dx):
            for j in range(Dy):
                if next_board.board[i][j] == Empty:
                    for symb in ['X', 'O']:
                        order_moves.append(Move(i, j, symb))

        threat_positions = []
        for move in order_moves:
            if move.valid(next_board):
                temp_board = deepcopy(next_board)
                temp_board.board[move.x][move.y] = move.symbol  #no turn change
                if temp_board.win():
                    threat_positions.append((move.x, move.y))

        if threat_positions:
            pos = threat_positions[0]
            for move in moves:
                if (move.x, move.y) == pos:
                    return move

    return None

In [ ]:
def pattern_based_playout(board, exploration_rate=0.2):
    board_copy = deepcopy(board)

    def evaluate_move(board, move):
        temp_board = deepcopy(board)
        temp_board.play(move)

        score = 1

        if board.turn == 'Order':
            directions = [(1, 0), (0, 1), (1, 1), (1, -1)]
            symbol = move.symbol

            for dx, dy in directions:
                consecutive = 1

                for i in range(1, 5):
                    nx, ny = move.x + dx*i, move.y + dy*i
                    if 0 <= nx < Dx and 0 <= ny < Dy and temp_board.board[nx][ny] == symbol:
                        consecutive += 1
                    else:
                        break

                for i in range(1, 5):
                    nx, ny = move.x - dx*i, move.y - dy*i
                    if 0 <= nx < Dx and 0 <= ny < Dy and temp_board.board[nx][ny] == symbol:
                        consecutive += 1
                    else:
                        break

                if consecutive >= 5:
                    score += 100
                elif consecutive == 4:
                    score += 10
                elif consecutive == 3:
                    score += 5
                elif consecutive == 2:
                    score += 2

        else:
            adjacent = [(0,1), (1,0), (1,1), (1,-1), (0,-1), (-1,0), (-1,-1), (-1,1)]
            for dx, dy in adjacent:
                nx, ny = move.x + dx, move.y + dy
                if 0 <= nx < Dx and 0 <= ny < Dy and temp_board.board[nx][ny] != Empty:
                    score += 2

        return score

    while not board_copy.terminal():
        moves = board_copy.legalMoves()
        if not moves:
            break

        if random.random() < exploration_rate:
            chosen_move = moves[random.randint(0, len(moves) - 1)]
        else:
            move_scores = []
            for move in moves:
                score = evaluate_move(board_copy, move)
                move_scores.append((move, score))

            chosen_move = max(move_scores, key=lambda x: x[1])[0]

        board_copy.play(chosen_move)

    return board_copy.score() if board_copy.terminal() else 0

In [ ]:
print("Testing improved playout methods:")
wins_heuristic = []
wins_pattern = []

for _ in range(100):
    game_h = Board()
    game_p = Board()

    result_h = heuristic_playout(game_h)
    result_p = pattern_based_playout(game_p)

    wins_heuristic.append(result_h)
    wins_pattern.append(result_p)

print(f"Heuristic playout win rate: {sum(wins_heuristic)/len(wins_heuristic):.4f}")
print(f"Pattern-based playout win rate: {sum(wins_pattern)/len(wins_pattern):.4f}")

## RAVE

In [ ]:
class RAVENode:
    def __init__(self, board, parent=None, move=None):
        self.board = deepcopy(board)
        self.parent = parent
        self.move = move
        self.children = []
        self.wins = 0
        self.visits = 0
        self.untried_moves = board.legalMoves()

        self.rave_wins = 0
        self.rave_visits = 0

    def select_child_rave(self, c=1.41, beta=0.5):
        if not self.children:
            return None

        best_value = -float('inf')
        best_child = None

        for child in self.children:
            if child.visits == 0:
                return child

            if child.visits > 0 and self.visits > 0:
                uct_score = (child.wins / child.visits) + c * math.sqrt(2 * math.log(self.visits) / child.visits)
            else:
                uct_score = float('inf')

            if child.rave_visits > 0:
                rave_score = child.rave_wins / child.rave_visits

                b = child.rave_visits / (child.visits + child.rave_visits + 4 * beta * beta * child.visits * child.rave_visits)

                score = (1 - b) * uct_score + b * rave_score
            else:
                score = uct_score

            if score > best_value:
                best_value = score
                best_child = child

        return best_child

    def add_child(self, move):
        child_board = deepcopy(self.board)
        child_board.play(move)
        child = RAVENode(child_board, parent=self, move=move)
        self.untried_moves.remove(move)
        self.children.append(child)
        return child

    def update(self, result):
        self.visits += 1
        self.wins += result

    def update_rave(self, result):
        self.rave_visits += 1
        self.rave_wins += result

    def is_fully_expanded(self):
        return len(self.untried_moves) == 0

    def is_terminal_node(self):
        return self.board.terminal()

In [ ]:
def rave_search(board, iterations=1000, playout_policy=pattern_based_playout):
    root = RAVENode(board)

    for _ in range(iterations):
        node = root
        board_state = deepcopy(node.board)
        visited_nodes = []
        action_playout = {}

        while node.is_fully_expanded() and not node.is_terminal_node():
            selected = node.select_child_rave()
            if selected is None:
                break
            node = selected
            if node.move:
                board_state.play(node.move)
                visited_nodes.append(node)
                action_key = (node.move.x, node.move.y, node.move.symbol)
                action_playout[action_key] = node

        if not node.is_terminal_node():
            if len(node.untried_moves) == 0:
                continue

            move = node.untried_moves[0]
            if len(node.untried_moves) > 1:
                idx = random.randint(0, len(node.untried_moves) - 1)
                move = node.untried_moves[idx]

            board_state.play(move)
            node = node.add_child(move)
            visited_nodes.append(node)
            action_key = (move.x, move.y, move.symbol)
            action_playout[action_key] = node

        result = playout_policy(board_state)

        for visited_node in visited_nodes:
            visited_node.update(result)

        for visited_node in visited_nodes:
            if visited_node.parent:
                for sibling in visited_node.parent.children:
                    action_key = (sibling.move.x, sibling.move.y, sibling.move.symbol)
                    if action_key in action_playout:
                        sibling.update_rave(result)

    if not root.children:
        moves = board.legalMoves()
        if not moves:
            return None
        if len(moves) == 1:
            return moves[0]
        return moves[random.randint(0, len(moves) - 1)]

    return sorted(root.children, key=lambda c: c.visits)[-1].move

## GRAVE

In [ ]:
class GRAVENode:
    def __init__(self, board, parent=None, move=None):
        self.board = deepcopy(board)
        self.parent = parent
        self.move = move
        self.children = []
        self.wins = 0
        self.visits = 0
        self.untried_moves = board.legalMoves()
        self.key = self._board_key()

    def _board_key(self):
        return str(self.board)

    def select_child_grave(self, amaf_wins, amaf_visits, c=1.41, beta=0.5):
        if not self.children:
            return None

        best_value = -float('inf')
        best_child = None

        for child in self.children:
            if child.visits == 0:
                return child

            if child.visits > 0 and self.visits > 0:
                uct_score = (child.wins / child.visits) + c * math.sqrt(2 * math.log(self.visits) / child.visits)
            else:
                uct_score = float('inf')

            action_key = (child.move.x, child.move.y, child.move.symbol)
            state_action_key = (self.key, action_key)

            if amaf_visits[state_action_key] > 0:
                rave_score = amaf_wins[state_action_key] / amaf_visits[state_action_key]

                b = amaf_visits[state_action_key] / (child.visits + amaf_visits[state_action_key] + 4 * beta * beta * child.visits * amaf_visits[state_action_key])

                score = (1 - b) * uct_score + b * rave_score
            else:
                score = uct_score

            if score > best_value:
                best_value = score
                best_child = child

        return best_child

    def add_child(self, move):
        child_board = deepcopy(self.board)
        child_board.play(move)
        child = GRAVENode(child_board, parent=self, move=move)
        self.untried_moves.remove(move)
        self.children.append(child)
        return child

    def update(self, result):
        self.visits += 1
        self.wins += result

    def is_fully_expanded(self):
        return len(self.untried_moves) == 0

    def is_terminal_node(self):
        return self.board.terminal()

In [ ]:
def grave_search(board, iterations=1000, playout_policy=pattern_based_playout):
    root = GRAVENode(board)

    amaf_wins = defaultdict(float)
    amaf_visits = defaultdict(float)

    for _ in range(iterations):
        node = root
        board_state = deepcopy(node.board)
        visited_states = []

        while node.is_fully_expanded() and not node.is_terminal_node():
            selected = node.select_child_grave(amaf_wins, amaf_visits)
            if selected is None:
                break
            node = selected
            if node.move:
                board_state.play(node.move)
                state_key = node.key
                action_key = (node.move.x, node.move.y, node.move.symbol)
                visited_states.append((state_key, action_key))

        if not node.is_terminal_node():
            if len(node.untried_moves) == 0:
                continue

            move = node.untried_moves[0]
            if len(node.untried_moves) > 1:
                idx = random.randint(0, len(node.untried_moves) - 1)
                move = node.untried_moves[idx]

            board_state.play(move)
            node = node.add_child(move)
            state_key = node.key
            action_key = (move.x, move.y, move.symbol)
            visited_states.append((state_key, action_key))

        result = playout_policy(board_state)

        current = node
        while current is not None:
            current.update(result)
            current = current.parent

        for state_key, action_key in visited_states:
            amaf_visits[(state_key, action_key)] += 1
            amaf_wins[(state_key, action_key)] += result

    if not root.children:
        moves = board.legalMoves()
        if not moves:
            return None
        if len(moves) == 1:
            return moves[0]
        return moves[random.randint(0, len(moves) - 1)]

    return sorted(root.children, key=lambda c: c.visits)[-1].move

## NRPA

In [ ]:
def nrpa_search(board, level=1, iterations=20):
    policy = defaultdict(float)

    def choose_move(board, policy):
        moves = board.legalMoves()
        if not moves:
            return None

        weights = []
        for move in moves:
            key = (move.x, move.y, move.symbol)
            weights.append(math.exp(policy[key]))

        total = sum(weights)
        if total == 0:
            return moves[random.randint(0, len(moves) - 1)]

        r = random.random() * total
        cumulative = 0
        for i, weight in enumerate(weights):
            cumulative += weight
            if cumulative > r:
                return moves[i]

        return moves[-1]

    def adapt_policy(policy, sequence, result):
        new_policy = defaultdict(float)
        for key in policy:
            new_policy[key] = policy[key]

        alpha = 0.1
        for move in sequence:
            key = (move.x, move.y, move.symbol)
            new_policy[key] += alpha * result

        return new_policy

    def playout(board, policy):
        sequence = []
        curr_board = deepcopy(board)

        while not curr_board.terminal():
            move = choose_move(curr_board, policy)
            if move is None:
                break
            sequence.append(move)
            curr_board.play(move)

        result = curr_board.score() if curr_board.terminal() else 0
        return sequence, result

    def nested_search(board, policy, level, n):
        if level == 0:
            return playout(board, policy)

        best_sequence = []
        best_score = -float('inf')

        for _ in range(n):
            sequence, score = nested_search(board, policy, level - 1, n)
            if score > best_score:
                best_score = score
                best_sequence = sequence

            policy = adapt_policy(policy, sequence, score)

        return best_sequence, best_score

    sequence, _ = nested_search(board, policy, level, iterations)

    if sequence:
        return sequence[0]

    moves = board.legalMoves()
    if not moves:
        return None
    return moves[random.randint(0, len(moves) - 1)]

## NMCS



In [ ]:
def nmcs_search(board, level=1):

    def random_playout(curr_board):
        board_copy = deepcopy(curr_board)

        while not board_copy.terminal():
            moves = board_copy.legalMoves()
            if not moves:
                break
            move = moves[random.randint(0, len(moves) - 1)]
            board_copy.play(move)

        return board_copy.score() if board_copy.terminal() else 0

    def nmcs(curr_board, level):
        if curr_board.terminal():
            return None, curr_board.score()

        if level == 0:
            moves = curr_board.legalMoves()
            if not moves:
                return None, 0

            best_move = moves[random.randint(0, len(moves) - 1)]
            next_board = deepcopy(curr_board)
            next_board.play(best_move)
            return best_move, random_playout(next_board)

        best_score = -float('inf')
        best_move = None

        for move in curr_board.legalMoves():
            next_board = deepcopy(curr_board)
            next_board.play(move)

            _, score = nmcs(next_board, level - 1)

            if score > best_score:
                best_score = score
                best_move = move

        return best_move, best_score

    best_move, _ = nmcs(deepcopy(board), level)

    if best_move:
        return best_move

    moves = board.legalMoves()
    if not moves:
        return None
    return moves[random.randint(0, len(moves) - 1)]

## Testing Framework for Algorithm Comparison

In [ ]:
def test_algorithm(algo_fn, num_games=20, opponent="random", algo_role="Order"):
    wins = 0
    avg_time = 0
    game_lengths = []

    for _ in range(num_games):
        game = Board()
        moves_count = 0
        algo_time = 0

        while not game.terminal() and moves_count < 100:
            curr_player = game.turn

            if (curr_player == 'Order' and algo_role == 'Order') or \
               (curr_player == 'Chaos' and algo_role == 'Chaos'):
                start_time = time.time()
                move = algo_fn(game)
                algo_time += time.time() - start_time

                if not isinstance(move, Move):
                    legal_moves = game.legalMoves()
                    if legal_moves:
                        move = legal_moves[random.randint(0, len(legal_moves) - 1)]
                    else:
                        break
            else:
                if opponent == "random":
                    moves = game.legalMoves()
                    if moves:
                        move = moves[random.randint(0, len(moves) - 1)]
                    else:
                        break
                elif opponent == "heuristic":
                    board_copy = deepcopy(game)
                    winning_move = find_tactical_move(board_copy)
                    if winning_move:
                        move = winning_move
                    else:
                        moves = game.legalMoves()
                        if moves:
                            move = moves[random.randint(0, len(moves) - 1)]
                        else:
                            break
                else:
                    opponent_move = opponent(game)
                    if not isinstance(opponent_move, Move):
                        moves = game.legalMoves()
                        if moves:
                            move = moves[random.randint(0, len(moves) - 1)]
                        else:
                            break
                    else:
                        move = opponent_move

            if isinstance(move, Move):
                game.play(move)
                moves_count += 1
            else:
                break

        if algo_role == 'Order':
            if game.win():
                wins += 1
        else:
            if not game.win():
                wins += 1

        avg_time += algo_time / num_games
        game_lengths.append(moves_count)

    win_rate = wins / num_games
    return {
        'win_rate': win_rate,
        'avg_time': avg_time,
        'avg_length': sum(game_lengths) / len(game_lengths)
    }

In [ ]:
def compare_algorithms(iterations=100):
    algorithms = {
        'UCT': lambda b: BestMoveUCT(b, iterations=iterations),
        'RAVE': lambda b: rave_search(b, iterations=iterations, playout_policy=pattern_based_playout),
        'GRAVE': lambda b: grave_search(b, iterations=iterations, playout_policy=pattern_based_playout),
        'NRPA': lambda b: nrpa_search(b, level=1, iterations=10),
        'NMCS': lambda b: nmcs_search(b, level=1),
        'Random': lambda b: b.legalMoves()[random.randint(0, len(b.legalMoves()) - 1)] if b.legalMoves() else None,
        'Heuristic': lambda b: find_tactical_move(b) or b.legalMoves()[random.randint(0, len(b.legalMoves()) - 1)] if b.legalMoves() else None
    }

    order_results = {}
    for name, algo in algorithms.items():
        print(f"Testing {name} as Order vs Random Chaos")
        order_results[name] = test_algorithm(algo, algo_role='Order')

    chaos_results = {}
    for name, algo in algorithms.items():
        print(f"Testing {name} as Chaos vs Random Order")
        chaos_results[name] = test_algorithm(algo, algo_role='Chaos')

    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    names = list(order_results.keys())
    win_rates = [result['win_rate'] for result in order_results.values()]
    plt.bar(names, win_rates)
    plt.title('Win Rates as Order vs Random Chaos')
    plt.ylim(0, 1)
    plt.ylabel('Win Rate')
    plt.xticks(rotation=45)

    plt.subplot(1, 2, 2)
    names = list(chaos_results.keys())
    win_rates = [result['win_rate'] for result in chaos_results.values()]
    plt.bar(names, win_rates)
    plt.title('Win Rates as Chaos vs Random Order')
    plt.ylim(0, 1)
    plt.xticks(rotation=45)

    plt.tight_layout()
    plt.savefig('algorithm_comparison.png')
    plt.show()

    return order_results, chaos_results

In [ ]:
compare_algorithms()

## Our algorithm

In [ ]:
# Inputs
Cp = 1.0
alpha = 0.8
gamma = 0.9
lbda = 0.8
Vinit = lambda s: 0.0
Vplayout = lambda s: 0.5

# Global tree structure: Memorized experience
tree = {}

def SARSA_UCT_ITERATION(board):
    episode = GENERATE_EPISODE(board)
    EXPAND_TREE(episode)
    BACKUP_TD_ERRORS(episode)

def GENERATE_EPISODE(board):
    episode = []
    b = deepcopy(board)
    while (True):
        if b.terminal():
            return episode
        if b.h in tree:
            move = UCB1_TREE_POLICY(b)
        else:  # Playout phase
            moves = b.legalMoves()
            rdm_move = random.randint(0, len(moves) - 1)
            move = moves[rdm_move]

        next_board = deepcopy(b)
        next_board.play(move)

        for _ in range(10):
            b_playout = deepcopy(next_board)
            res = b_playout.playout()
            res += (1/10)* (1 - res)

        episode.append((next_board, res))
        b = next_board

def EXPAND_TREE(episode):
    Boardnew = next((board for board, _ in episode if board.h not in tree), None)
    if Boardnew is not None:
        tree[Boardnew.h] = {'V': Vinit(Boardnew), 'n': 0}

def BACKUP_TD_ERRORS(episode):
    TD_errors = 0
    Vnext = 0
    for board, reward in reversed(episode):
        if board.h in tree:
            Vcurrent = tree[board.h]['V']
        else:
            Vcurrent = Vplayout(board)

        err = reward + gamma * Vnext - Vcurrent  # TD error
        TD_errors = lbda * gamma * TD_errors + err  # accumulate and decay TD error

        if board.h in tree:
            tree[board.h]['n'] += 1
            # Update value using a MC-like step-size
            alpha = 1 / tree[board.h]['n']
            tree[board.h]['V'] += alpha * TD_errors

        Vnext = Vcurrent

def UCB1_TREE_POLICY(board:Board):
    max_qucb = -float('inf')
    best_move = None

    for move in board.legalMoves():
        b = deepcopy(board)
        b.play(move)

        if b.h in tree:
            Vnorm = normalize(tree[b.h]['V'])
            QUCB = Vnorm + Cp * math.sqrt(math.log(tree[board.h]['n']) / tree[b.h]['n'])
        else:
            QUCB = float('inf')

        if QUCB > max_qucb:
            max_qucb = QUCB
            best_move = move

    return best_move

def SIMULATE_TRANSITION(board, move):
    b = deepcopy(board)
    b.play(move)
    res = 0
    for _ in range(10):
        b_playout = deepcopy(b)
        res = b_playout.playout()
        res += (1/10)* (1 - res)
    return b, res

def normalize(value):
    return min(1, max(0, value))  # Simple clamping to [0, 1]


def SARSA_UCT_BestMove(board, iterations=100):
    best_val = -float('inf')
    best = None
    for i in range(iterations):
        SARSA_UCT_ITERATION(board)

    for move in board.legalMoves():
        next_board = deepcopy(board)
        next_board.play(move)
        if next_board.h in tree:
            val = tree[next_board.h]['V']
        else:
            val = Vinit(next_board)
        if val > best_val:
            best_val = val
            best = move

    return best

In [ ]:
def train_SARSA_UCT(iterations=100, opponent="SARSA_UCT", opponent_name='SARSA_UCT'):

    algorithms = {
        'NMCS': lambda b: nmcs_search(b, level=1),
        'Random': lambda b: b.legalMoves()[random.randint(0, len(b.legalMoves()) - 1)] if b.legalMoves() else None,
        'Heuristic': lambda b: find_tactical_move(b) or b.legalMoves()[random.randint(0, len(b.legalMoves()) - 1)] if b.legalMoves() else None
    }

    opponent = lambda b: SARSA_UCT_BestMove(b, iterations=iterations)

    order_results = {}
    for name, algo in algorithms.items():
        print(f"Testing {name} as Order vs {opponent_name} Chaos")
        order_results[name] = test_algorithm(algo, num_games=1, opponent=opponent, algo_role='Order')


In [ ]:
train_SARSA_UCT()

In [ ]:
def compare_SARSA_UCT(iterations=100, opponent="SARSA_UCT", opponent_name='SARSA_UCT'):
    opponent_name = str(opponent)

    algorithms = {
        'NMCS': lambda b: nmcs_search(b, level=1),
        'Random': lambda b: b.legalMoves()[random.randint(0, len(b.legalMoves()) - 1)] if b.legalMoves() else None,
        'Heuristic': lambda b: find_tactical_move(b) or b.legalMoves()[random.randint(0, len(b.legalMoves()) - 1)] if b.legalMoves() else None
    }

    opponent = lambda b: SARSA_UCT_BestMove(b, iterations=1)

    order_results = {}
    for name, algo in algorithms.items():
        print(f"Testing {name} as Order vs {opponent_name} Chaos")
        order_results[name] = test_algorithm(algo, num_games=50, opponent=opponent, algo_role='Order')

    chaos_results = {}
    for name, algo in algorithms.items():
        print(f"Testing {name} as Chaos vs {opponent_name} Order")
        chaos_results[name] = test_algorithm(algo, num_games=50, opponent=opponent, algo_role='Chaos')

    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    names = list(order_results.keys())
    win_rates = [result['win_rate'] for result in order_results.values()]
    plt.bar(names, win_rates)
    plt.title(f'Win Rates as Order vs {opponent_name} Chaos')
    plt.ylim(0, 1)
    plt.ylabel('Win Rate')
    plt.xticks(rotation=45)

    plt.subplot(1, 2, 2)
    names = list(chaos_results.keys())
    win_rates = [result['win_rate'] for result in chaos_results.values()]
    plt.bar(names, win_rates)
    plt.title(f'Win Rates as Chaos vs {opponent_name} Order')
    plt.ylim(0, 1)
    plt.xticks(rotation=45)

    plt.tight_layout()
    plt.savefig(f'algorithm_comparison_{opponent_name}.png')
    plt.show()

    return order_results, chaos_results

In [ ]:
compare_SARSA_UCT()

# Testing GUI

The interface enables to:



1. Compare different algorithms (UCT, RAVE, GRAVE, NRPA, NMCS) against randomm or heuristic opponents
2. Test algorithms in direct matchups against each other
3. Analyze algorithm sensitivity to parameter changes
4. Test algorithm performance from specific board positions



In [ ]:

display(HTML("""
<style>
    .widget-label {
        font-weight: bold;
        color: #2c3e50;
    }
    .mcts-container {
        background-color: #f8f9fa;
        border-radius: 8px;
        padding: 15px;
        box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
        margin-bottom: 15px;
    }
    .mcts-header {
        text-align: center;
        font-size: 24px;
        font-weight: bold;
        color: #3498db;
        margin-bottom: 20px;
        border-bottom: 2px solid #eaeaea;
        padding-bottom: 10px;
    }
    .mcts-section {
        background-color: white;
        border-radius: 6px;
        padding: 12px;
        margin-bottom: 12px;
        border: 1px solid #e0e0e0;
    }
    .mcts-section-title {
        font-size: 18px;
        font-weight: bold;
        color: #2c3e50;
        margin-bottom: 10px;
    }
    .alert-info {
        background-color: #d1ecf1;
        color: #0c5460;
        padding: 10px;
        border-radius: 5px;
        border-left: 5px solid #17a2b8;
    }
    .alert-warning {
        background-color: #fff3cd;
        color: #856404;
        padding: 10px;
        border-radius: 5px;
        border-left: 5px solid #ffc107;
    }
    .alert-danger {
        background-color: #f8d7da;
        color: #721c24;
        padding: 10px;
        border-radius: 5px;
        border-left: 5px solid #dc3545;
    }
    .alert-success {
        background-color: #d4edda;
        color: #155724;
        padding: 10px;
        border-radius: 5px;
        border-left: 5px solid #28a745;
    }
    .button-panel {
        display: flex;
        gap: 10px;
        margin-top: 15px;
        margin-bottom: 15px;
    }
</style>
"""))


In [ ]:
display(HTML("""
<style>
    .jupyter-widgets-output-area {
        padding: 15px;
    }

    .widget-tab {
        border-radius: 4px;
        overflow: hidden;
        box-shadow: 0 2px 5px rgba(0,0,0,0.1);
    }

    .widget-tab-contents {
        padding: 10px;
        border: 1px solid #eaeaea;
        border-top: none;
    }

    .jupyter-button {
        transition: all 0.3s ease;
    }

    .jupyter-button:hover {
        transform: translateY(-2px);
        box-shadow: 0 4px 8px rgba(0,0,0,0.1);
    }

    .game-board-container {
        background-color: white;
        border-radius: 8px;
        box-shadow: 0 4px 8px rgba(0,0,0,0.1);
        padding: 15px;
        margin: 15px 0;
    }

    .dataframe {
        border-collapse: collapse;
        width: 100%;
        max-width: 800px;
        margin: 0 auto;
        font-size: 14px;
    }

    .dataframe th {
        background-color: #3498db;
        color: white;
        font-weight: bold;
        padding: 12px 8px;
        text-align: center;
    }

    .dataframe td {
        padding: 10px 8px;
        text-align: center;
        border-bottom: 1px solid #ddd;
    }

    .dataframe tr:nth-child(even) {
        background-color: #f9f9f9;
    }

    .dataframe tr:hover {
        background-color: #f1f1f1;
    }
</style>
"""))

In [ ]:
def test_algorithm(algo_fn, num_games=20, opponent="random", algo_role="Order"):
    """Test an algorithm """
    wins = 0
    avg_time = 0
    game_lengths = []

    for _ in range(num_games):
        try:
            game = Board()
            moves_count = 0
            algo_time = 0

            while not game.terminal() and moves_count < 100:
                curr_player = game.turn

                if (curr_player == 'Order' and algo_role == 'Order') or \
                   (curr_player == 'Chaos' and algo_role == 'Chaos'):
                    try:
                        start_time = time.time()

                        try:
                            move = algo_fn(deepcopy(game))
                        except Exception as algo_error:
                            print(f"Algorithm error: {str(algo_error)}")
                            legal_moves = game.legalMoves()
                            move = legal_moves[random.randint(0, len(legal_moves) - 1)] if legal_moves else None

                        algo_time += time.time() - start_time

                        if not isinstance(move, Move):
                            legal_moves = game.legalMoves()
                            if legal_moves:
                                move = legal_moves[random.randint(0, len(legal_moves) - 1)]
                            else:
                                break
                    except Exception as move_error:
                        print(f"Move selection error: {str(move_error)}")
                        legal_moves = game.legalMoves()
                        if legal_moves:
                            move = legal_moves[random.randint(0, len(legal_moves) - 1)]
                        else:
                            break
                else:
                    if opponent == "random":
                        moves = game.legalMoves()
                        if moves:
                            move = moves[random.randint(0, len(moves) - 1)]
                        else:
                            break
                    elif opponent == "heuristic":
                        board_copy = deepcopy(game)
                        winning_move = find_tactical_move(board_copy)
                        if winning_move:
                            move = winning_move
                        else:
                            moves = game.legalMoves()
                            if moves:
                                move = moves[random.randint(0, len(moves) - 1)]
                            else:
                                break
                    else:
                        moves = game.legalMoves()
                        if moves:
                            move = moves[random.randint(0, len(moves) - 1)]
                        else:
                            break

                game.play(move)
                moves_count += 1

            if algo_role == 'Order':
                if game.win():
                    wins += 1
            else:
                if not game.win():
                    wins += 1

            avg_time += algo_time / num_games
            game_lengths.append(moves_count)

        except Exception as game_error:
            print(f"Game simulation error: {str(game_error)}")
            continue

    win_rate = wins / num_games if num_games > 0 else 0
    avg_length = sum(game_lengths) / len(game_lengths) if game_lengths else 0

    return {
        'win_rate': win_rate,
        'avg_time': avg_time,
        'avg_length': avg_length
    }

In [ ]:
def create_game_replay(algorithm_name, opponent_name, role, num_iterations=100):
    """Creates an animated replay of a game between the algorithm and opponent"""
    game = Board()
    moves_history = []

    if algorithm_name == 'UCT':
        algo_func = lambda b: BestMoveUCT(b, iterations=num_iterations)
    elif algorithm_name == 'RAVE':
        algo_func = lambda b: rave_search(b, iterations=num_iterations, playout_policy=pattern_based_playout)
    elif algorithm_name == 'GRAVE':
        algo_func = lambda b: grave_search(b, iterations=num_iterations, playout_policy=pattern_based_playout)
    elif algorithm_name == 'NRPA':
        algo_func = lambda b: nrpa_search(b, level=1, iterations=10)
    elif algorithm_name == 'NMCS':
        algo_func = lambda b: nmcs_search(b, level=1)
    elif algorithm_name == 'Random':
        algo_func = lambda b: b.legalMoves()[random.randint(0, len(b.legalMoves()) - 1)] if b.legalMoves() else None
    elif algorithm_name == 'Heuristic':
        algo_func = lambda b: find_tactical_move(b) or (b.legalMoves()[random.randint(0, len(b.legalMoves()) - 1)] if b.legalMoves() else None)

    if opponent_name == 'random':
        opponent_func = lambda b: b.legalMoves()[random.randint(0, len(b.legalMoves()) - 1)] if b.legalMoves() else None
    elif opponent_name == 'heuristic':
        opponent_func = lambda b: find_tactical_move(b) or (b.legalMoves()[random.randint(0, len(b.legalMoves()) - 1)] if b.legalMoves() else None)
    else:
        opponent_func = lambda b: b.legalMoves()[random.randint(0, len(b.legalMoves()) - 1)] if b.legalMoves() else None

    while not game.terminal() and len(moves_history) < 36:
        current_player = game.turn
        move = None

        if (current_player == 'Order' and role == 'Order') or (current_player == 'Chaos' and role == 'Chaos'):
            move = algo_func(deepcopy(game))
        else:
            move = opponent_func(deepcopy(game))

        if move and move.valid(game):
            moves_history.append((deepcopy(game.board), move, current_player))
            game.play(move)
        else:
            break

    if moves_history:
        moves_history.append((deepcopy(game.board), None, None))

    def create_state_display(state_idx):
        board, move, player = moves_history[state_idx]

        plt.figure(figsize=(12, 8))

        plt.subplot(1, 2, 1)
        create_board_visualization(board, move)

        plt.subplot(1, 2, 2)
        plt.axis('off')

        title = f"Game Replay: {algorithm_name} ({role}) vs {opponent_name}"
        plt.title(title, fontsize=16, fontweight='bold')

        info_text = f"Move {state_idx}/{len(moves_history)-1}\n\n"

        if state_idx < len(moves_history) - 1:
            next_board, next_move, next_player = moves_history[state_idx]
            if next_move:
                player_str = f"{next_player} (Algorithm)" if ((next_player == 'Order' and role == 'Order') or
                                                             (next_player == 'Chaos' and role == 'Chaos')) else f"{next_player} (Opponent)"
                info_text += f"Player: {player_str}\n"
                info_text += f"Symbol: {next_move.symbol}\n"
                info_text += f"Position: ({next_move.x}, {next_move.y})\n\n"

        if state_idx == len(moves_history) - 1:
            if game.win():
                winner = "Order"
                status = f"Game Over: {winner} wins!"
                if (winner == role):
                    status += "\nAlgorithm wins!"
                else:
                    status += "\nOpponent wins!"
            else:
                status = "Game Over: Draw (Chaos wins)"
                if (role == "Chaos"):
                    status += "\nAlgorithm wins!"
                else:
                    status += "\nOpponent wins!"
            info_text += status

        plt.text(0.1, 0.5, info_text, fontsize=14, verticalalignment='center')
        plt.tight_layout()

    create_state_display(0)

    max_states = len(moves_history) - 1
    slider = widgets.IntSlider(
        value=0,
        min=0,
        max=max_states,
        step=1,
        description='Move:',
        continuous_update=False,
        orientation='horizontal',
        layout=widgets.Layout(width='80%')
    )

    play_button = widgets.Button(
        description='Play',
        disabled=False,
        button_style='success',
        tooltip='Play the game animation',
        icon='play'
    )

    output = widgets.Output()

    def update_display(change):
        with output:
            clear_output(wait=True)
            create_state_display(slider.value)
            plt.show()

    slider.observe(update_display, names='value')

    def play_animation(b):
        play_button.disabled = True
        play_button.description = 'Playing...'

        for i in range(slider.value, slider.max + 1):
            slider.value = i
            time.sleep(1)

        play_button.disabled = False
        play_button.description = 'Play'

    play_button.on_click(lambda b: play_animation(b))

    with output:
        create_state_display(0)
        plt.show()

    controls = widgets.HBox([slider, play_button])
    return widgets.VBox([controls, output])

In [ ]:
def create_mcts_testing_ui():
    header = widgets.HTML(value="<div class='mcts-header'>Monte Carlo Tree Search Algorithm Testing</div>")

    algo_section = widgets.HTML(value="<div class='mcts-section-title'>Select Algorithms</div>")
    uct_check = widgets.Checkbox(value=True, description='UCT')
    rave_check = widgets.Checkbox(value=True, description='RAVE')
    grave_check = widgets.Checkbox(value=True, description='GRAVE')
    nrpa_check = widgets.Checkbox(value=False, description='NRPA')
    nmcs_check = widgets.Checkbox(value=False, description='NMCS')
    random_check = widgets.Checkbox(value=False, description='Random')
    heuristic_check = widgets.Checkbox(value=False, description='Heuristic')

    algo_container = widgets.VBox([
        algo_section,
        widgets.VBox([uct_check, rave_check, grave_check, nrpa_check, nmcs_check, random_check, heuristic_check])
    ], layout=widgets.Layout(margin='10px 0px'))

    algo_box = widgets.Box([algo_container],
                          layout=widgets.Layout(margin='0px 0px 10px 0px'))
    algo_box.add_class('mcts-section')

    param_section = widgets.HTML(value="<div class='mcts-section-title'>Test Parameters</div>")
    games_slider = widgets.IntSlider(
        value=10,
        min=5,
        max=50,
        step=5,
        description='Games:',
        style={'description_width': 'initial'}
    )
    iterations_slider = widgets.IntSlider(
        value=100,
        min=50,
        max=500,
        step=50,
        description='Iterations:',
        style={'description_width': 'initial'}
    )

    opponent_dropdown = widgets.Dropdown(
        options=['random', 'heuristic'],
        value='random',
        description='Opponent:',
        style={'description_width': 'initial'}
    )

    role_radio = widgets.RadioButtons(
        options=['Order', 'Chaos', 'Both'],
        value='Both',
        description='Test as:',
        style={'description_width': 'initial'}
    )

    param_container = widgets.VBox([
        param_section,
        games_slider,
        iterations_slider,
        opponent_dropdown,
        role_radio
    ], layout=widgets.Layout(margin='10px 0px'))

    param_box = widgets.Box([param_container],
                          layout=widgets.Layout(margin='0px 0px 10px 0px'))
    param_box.add_class('mcts-section')

    run_button = widgets.Button(
        description='Run Tests',
        button_style='success',
        icon='play',
        layout=widgets.Layout(width='150px', height='40px')
    )

    clear_button = widgets.Button(
        description='Clear Results',
        button_style='danger',
        icon='trash',
        layout=widgets.Layout(width='150px', height='40px')
    )

    button_html = widgets.HTML(value="<div class='button-panel'>")
    control_box = widgets.HBox([run_button, clear_button])

    status_area = widgets.HTML(value="<div class='alert-info'>Ready to run tests</div>")
    results_area = widgets.Output()

    left_panel = widgets.VBox([algo_box])
    right_panel = widgets.VBox([param_box])
    top_panels = widgets.HBox([left_panel, right_panel])

    main_container = widgets.VBox([
        header,
        top_panels,
        control_box,
        status_area,
        results_area
    ])

    main_box = widgets.Box([main_container])
    main_box.add_class('mcts-container')

    def on_run_button_clicked(b):
      with results_area:
          clear_output()

      selected_algos = []
      if uct_check.value:
          selected_algos.append('UCT')
      if rave_check.value:
          selected_algos.append('RAVE')
      if grave_check.value:
          selected_algos.append('GRAVE')
      if nrpa_check.value:
          selected_algos.append('NRPA')
      if nmcs_check.value:
          selected_algos.append('NMCS')
      if random_check.value:
          selected_algos.append('Random')
      if heuristic_check.value:
          selected_algos.append('Heuristic')

      if not selected_algos:
          status_area.value = "<div class='alert-danger'>Please select at least one algorithm</div>"
          return

      num_games = games_slider.value
      iterations = iterations_slider.value
      opponent = opponent_dropdown.value
      role = role_radio.value

      status_area.value = f"<div class='alert-warning'>Running tests for {', '.join(selected_algos)}...</div>"

      try:
          algo_funcs = {}

          for algo in selected_algos:
              if algo == 'UCT':
                  algo_funcs[algo] = lambda b, it=iterations: BestMoveUCT(b, iterations=it)
              elif algo == 'RAVE':
                  algo_funcs[algo] = lambda b, it=iterations: rave_search(b, iterations=it, playout_policy=pattern_based_playout)
              elif algo == 'GRAVE':
                  algo_funcs[algo] = lambda b, it=iterations: grave_search(b, iterations=it, playout_policy=pattern_based_playout)
              elif algo == 'NRPA':
                  algo_funcs[algo] = lambda b: nrpa_search(b, level=1, iterations=10)
              elif algo == 'NMCS':
                  algo_funcs[algo] = lambda b: nmcs_search(b, level=1)
              elif algo == 'Random':
                  algo_funcs[algo] = lambda b: b.legalMoves()[random.randint(0, len(b.legalMoves()) - 1)] if b.legalMoves() else None
              elif algo == 'Heuristic':
                  algo_funcs[algo] = lambda b: find_tactical_move(b) or (b.legalMoves()[random.randint(0, len(b.legalMoves()) - 1)] if b.legalMoves() else None)

          results = {}
          test_roles = role.lower()

          if test_roles == 'order' or test_roles == 'both':
              for algo_name, algo_func in algo_funcs.items():
                  status_area.value = f"<div class='alert-warning'>Testing {algo_name} as Order...</div>"
                  try:
                      results[algo_name + '_Order'] = test_algorithm(algo_func, num_games=num_games, opponent=opponent, algo_role="Order")
                  except Exception as algo_error:
                      status_area.value = f"<div class='alert-danger'>Error testing {algo_name} as Order: {str(algo_error)}</div>"
                      return

          if test_roles == 'chaos' or test_roles == 'both':
              for algo_name, algo_func in algo_funcs.items():
                  status_area.value = f"<div class='alert-warning'>Testing {algo_name} as Chaos...</div>"
                  try:
                      results[algo_name + '_Chaos'] = test_algorithm(algo_func, num_games=num_games, opponent=opponent, algo_role="Chaos")
                  except Exception as algo_error:
                      status_area.value = f"<div class='alert-danger'>Error testing {algo_name} as Chaos: {str(algo_error)}</div>"
                      return

          data = []
          for key, res in results.items():
              algo_name, role_name = key.split('_')
              data.append({
                  'Algorithm': algo_name,
                  'Role': role_name,
                  'Win Rate': res['win_rate'],
                  'Avg Time': res['avg_time'],
                  'Avg Length': res['avg_length']
              })

          df = pd.DataFrame(data)

          with results_area:
              clear_output()

              plt.style.use('seaborn-v0_8-whitegrid')

              tabs = widgets.Tab()
              tab_contents = []

              charts_tab = widgets.Output()
              with charts_tab:
                  plt.figure(figsize=(10, 6))
                  ax = sns.barplot(x='Algorithm', y='Win Rate', hue='Role', data=df, palette='viridis')
                  plt.title('Win Rates by Algorithm and Role', fontsize=16, fontweight='bold')
                  plt.xlabel('Algorithm', fontsize=12)
                  plt.ylabel('Win Rate', fontsize=12)
                  plt.ylim(0, 1)

                  for container in ax.containers:
                      ax.bar_label(container, fmt='%.2f', fontsize=10)

                  plt.legend(title='Role', title_fontsize=12)
                  plt.tight_layout()
                  plt.show()

                  plt.figure(figsize=(10, 6))
                  ax = sns.barplot(x='Algorithm', y='Avg Time', hue='Role', data=df, palette='viridis')
                  plt.title('Average Computation Time (seconds)', fontsize=16, fontweight='bold')
                  plt.xlabel('Algorithm', fontsize=12)
                  plt.ylabel('Time (seconds)', fontsize=12)

                  for container in ax.containers:
                      ax.bar_label(container, fmt='%.3f', fontsize=10)

                  plt.legend(title='Role', title_fontsize=12)
                  plt.tight_layout()
                  plt.show()

                  plt.figure(figsize=(10, 6))
                  ax = sns.barplot(x='Algorithm', y='Avg Length', hue='Role', data=df, palette='viridis')
                  plt.title('Average Game Length (moves)', fontsize=16, fontweight='bold')
                  plt.xlabel('Algorithm', fontsize=12)
                  plt.ylabel('Number of Moves', fontsize=12)

                  for container in ax.containers:
                      ax.bar_label(container, fmt='%.1f', fontsize=10)

                  plt.legend(title='Role', title_fontsize=12)
                  plt.tight_layout()
                  plt.show()

                  display(HTML("<h3 style='color: #2c3e50; margin-top: 20px;'>Results Summary</h3>"))

                  display_df = df.copy()
                  display_df['Win Rate'] = display_df['Win Rate'].map(lambda x: f"{x:.2%}")
                  display_df['Avg Time'] = display_df['Avg Time'].map(lambda x: f"{x:.3f}s")
                  display_df['Avg Length'] = display_df['Avg Length'].map(lambda x: f"{x:.1f}")

                  display(display_df.style.set_properties(**{
                      'text-align': 'center',
                      'border': '1px solid #ddd',
                      'padding': '8px'
                  }).set_table_styles([
                      {'selector': 'th', 'props': [('background-color', '#3498db'),
                                                ('color', 'white'),
                                                ('font-weight', 'bold'),
                                                ('text-align', 'center'),
                                                ('padding', '8px')]},
                      {'selector': 'tr:nth-of-type(odd)', 'props': [('background-color', '#f8f9fa')]},
                      {'selector': 'tr:hover', 'props': [('background-color', '#e9ecef')]}
                  ]))

              game_viz_tab = widgets.Output()
              with game_viz_tab:
                  if selected_algos:
                      algo_dropdown = widgets.Dropdown(
                          options=selected_algos,
                          value=selected_algos[0],
                          description='Algorithm:',
                          style={'description_width': 'initial'}
                      )

                      role_dropdown = widgets.Dropdown(
                          options=['Order', 'Chaos'],
                          value='Order' if role.lower() != 'chaos' else 'Chaos',
                          description='Role:',
                          style={'description_width': 'initial'}
                      )

                      viz_button = widgets.Button(
                          description='Show Game',
                          button_style='success',
                          tooltip='Show a game visualization'
                      )

                      viz_output = widgets.Output()

                      def on_viz_button_clicked(b):
                          with viz_output:
                              clear_output()
                              display(HTML("<div style='text-align:center; padding:10px;'><h3>Loading game replay...</h3></div>"))
                              try:
                                  replay = create_game_replay(
                                      algorithm_name=algo_dropdown.value,
                                      opponent_name=opponent,
                                      role=role_dropdown.value,
                                      num_iterations=iterations
                                  )
                                  clear_output()
                                  display(replay)
                              except Exception as e:
                                  clear_output()
                                  display(HTML(f"<div class='alert-danger'>Error creating game replay: {str(e)}</div>"))

                      viz_button.on_click(on_viz_button_clicked)

                      controls = widgets.HBox([algo_dropdown, role_dropdown, viz_button])
                      display(controls)
                      display(viz_output)

                      display(HTML("<h3 style='margin-top:20px;'>Sample Board Visualization</h3>"))
                      sample_board = Board()
                      sample_moves = [
                          Move(0, 0, 'X'), Move(0, 1, 'O'), Move(1, 1, 'X'),
                          Move(2, 2, 'O'), Move(3, 3, 'X'), Move(4, 4, 'O')
                      ]
                      for move in sample_moves:
                          sample_board.play(move)

                      fig = create_board_visualization(sample_board, sample_moves[-1])
                      plt.show()
                  else:
                      display(HTML("<div class='alert-warning'>Please select at least one algorithm to visualize</div>"))

              raw_data_tab = widgets.Output()
              with raw_data_tab:
                  download_button = widgets.Button(
                      description='Download Results CSV',
                      button_style='info',
                      tooltip='Download the results as a CSV file'
                  )

                  def create_csv_content(df):
                      return df.to_csv(index=False)

                  def create_download_link(csv_content, filename="results.csv"):
                      b64 = base64.b64encode(csv_content.encode())
                      payload = b64.decode()
                      html = f'''
                      <a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">
                          <button style="background-color: #4CAF50; color: white; padding: 10px 15px; border: none; border-radius: 4px; cursor: pointer;">
                              Download CSV File
                          </button>
                      </a>
                      '''
                      return HTML(html)

                  def on_download_button_clicked(b):
                      csv_content = create_csv_content(df)
                      display(create_download_link(csv_content))

                  download_button.on_click(on_download_button_clicked)
                  display(download_button)

                  display(HTML("<h3 style='margin-top:20px;'>Raw Results Data</h3>"))
                  display(df)

                  if len(df) > 2:
                      display(HTML("<h3 style='margin-top:20px;'>Correlation Between Metrics</h3>"))
                      plt.figure(figsize=(8, 6))
                      corr_df = df[['Win Rate', 'Avg Time', 'Avg Length']].corr()
                      sns.heatmap(corr_df, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0)
                      plt.title('Correlation Between Metrics', fontsize=14)
                      plt.tight_layout()
                      plt.show()
              tab_contents = [charts_tab, game_viz_tab, raw_data_tab]
              tabs.children = tab_contents
              tabs.set_title(0, 'Charts & Summary')
              tabs.set_title(1, 'Game Visualization')
              tabs.set_title(2, 'Raw Data & Export')

              display(tabs)

          status_area.value = "<div class='alert-success'>Testing complete!</div>"

      except Exception as e:
          status_area.value = f"<div class='alert-danger'>Error during testing: {str(e)}</div>"
          with results_area:
              clear_output()
              print(f"Detailed error information: {str(e)}")

    def on_clear_button_clicked(b):
        with results_area:
            clear_output()
        status_area.value = "<div class='alert-info'>Results cleared</div>"

    run_button.on_click(on_run_button_clicked)
    clear_button.on_click(on_clear_button_clicked)

    return main_box

In [ ]:
def create_board_visualization(board_state=None, last_move=None, highlighted_cells=None):
    """
    Creates a visualization of the Order and Chaos game board

    Parameters:
    - board_state: A Board object or 2D array representing the current state
    - last_move: The last move played (to highlight)
    - highlighted_cells: List of (x,y) coordinates to highlight (for showing potential moves)

    Returns:
    - Matplotlib figure object
    """
    fig, ax = plt.subplots(figsize=(8, 8))
    fig.patch.set_facecolor('white')

    if board_state is None:
        board_data = np.array([['-' for _ in range(Dx)] for _ in range(Dy)])
    elif isinstance(board_state, Board):
        board_data = board_state.board
    else:
        board_data = board_state

    for i in range(Dx + 1):
        ax.axhline(i, color='black', linewidth=2)
        ax.axvline(i, color='black', linewidth=2)

    colors = {'X': '#3498db', 'O': '#e74c3c', '-': 'white'}

    for i in range(Dx):
        for j in range(Dy):
            cell_value = board_data[i][j]
            if cell_value != '-':
                color = colors[cell_value]
                if cell_value == 'X':
                    ax.plot([j+0.2, j+0.8], [i+0.2, i+0.8], color=color, linewidth=10, alpha=0.8)
                    ax.plot([j+0.2, j+0.8], [i+0.8, i+0.2], color=color, linewidth=10, alpha=0.8)
                else:
                    circle = plt.Circle((j+0.5, i+0.5), 0.3, color=color, alpha=0.8)
                    ax.add_patch(circle)

    if last_move is not None:
        x, y, symbol = last_move.x, last_move.y, last_move.symbol
        rect = plt.Rectangle((y, x), 1, 1, fill=True, alpha=0.2, color='yellow')
        ax.add_patch(rect)

    if highlighted_cells is not None:
        for x, y in highlighted_cells:
            rect = plt.Rectangle((y, x), 1, 1, fill=True, alpha=0.2, color='green')
            ax.add_patch(rect)

    ax.set_xlim(0, Dx)
    ax.set_ylim(Dy, 0)  # Inverting y-axis to match grid coordinates
    ax.set_xticks([])
    ax.set_yticks([])

    for i in range(Dx):
        ax.text(-0.2, i+0.5, str(i), ha='center', va='center', fontsize=12, fontweight='bold')
        ax.text(i+0.5, -0.2, str(i), ha='center', va='center', fontsize=12, fontweight='bold')

    ax.set_title('Order and Chaos Game Board', fontsize=16, fontweight='bold')

    x_marker = plt.Line2D([], [], color=colors['X'], marker='x', linestyle='None',
                          markersize=15, markeredgewidth=3, label='X')
    o_marker = plt.Line2D([], [], color=colors['O'], marker='o', linestyle='None',
                          markersize=15, label='O')
    last_move_marker = plt.Line2D([], [], color='yellow', marker='s', linestyle='None',
                                 markersize=15, alpha=0.5, label='Last Move')

    ax.legend(handles=[x_marker, o_marker, last_move_marker],
              loc='upper center', bbox_to_anchor=(0.5, -0.05),
              ncol=3, frameon=False)

    plt.tight_layout()
    return fig

In [ ]:
main_ui = create_mcts_testing_ui()
display(main_ui)